# <center>CIFAR-100</center>

<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Librairies-Import" data-toc-modified-id="Librairies-Import-1">Librairies Import</a></span></li><li><span><a href="#Dataset-Loading" data-toc-modified-id="Dataset-Loading-2">Dataset Loading</a></span></li><li><span><a href="#Normalization" data-toc-modified-id="Normalization-3">Normalization</a></span></li><li><span><a href="#DataModule" data-toc-modified-id="DataModule-4">DataModule</a></span></li><li><span><a href="#Model" data-toc-modified-id="Model-5">Model</a></span></li><li><span><a href="#Lightning-Pipeline" data-toc-modified-id="Lightning-Pipeline-6">Lightning Pipeline</a></span></li><li><span><a href="#Model-Training-and-Evaluation" data-toc-modified-id="Model-Training-and-Evaluation-7">Model Training and Evaluation</a></span></li></ul></div>

## Librairies Import

In [1]:
import os
import warnings
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import torch
import torch.nn.functional as F
from torch.optim.lr_scheduler import OneCycleLR, ReduceLROnPlateau
import torchvision
from torchvision.utils import make_grid
from torchmetrics import ConfusionMatrix
from torchmetrics.functional import accuracy
from pytorch_lightning import Trainer
from pytorch_lightning.callbacks import ModelCheckpoint, LearningRateMonitor, EarlyStopping
from pytorch_lightning.loggers import TensorBoardLogger
from pytorch_lightning import LightningModule

## Dataset Loading

We start by downloading the train and test sets of CIFAR-100.

In [3]:
train_set = torchvision.datasets.CIFAR100(root="./data", download=True, train=True, transform=train_transform)
test_set = torchvision.datasets.CIFAR100(root="./data", download=True, train=False, transform=test_transform)

Files already downloaded and verified
Files already downloaded and verified


We then calculate the number of images by class for each set.

In [6]:
train_classes_counts = {}

for image in train_set:
    label = train_set.classes[image[1]]
    if label not in train_classes_counts:
        train_classes_counts[label] = 1
    else:
        train_classes_counts[label] += 1

train_classes_counts

{'cattle': 500,
 'dinosaur': 500,
 'apple': 500,
 'boy': 500,
 'aquarium_fish': 500,
 'telephone': 500,
 'train': 500,
 'cup': 500,
 'cloud': 500,
 'elephant': 500,
 'keyboard': 500,
 'willow_tree': 500,
 'sunflower': 500,
 'castle': 500,
 'sea': 500,
 'bicycle': 500,
 'wolf': 500,
 'squirrel': 500,
 'shrew': 500,
 'pine_tree': 500,
 'rose': 500,
 'television': 500,
 'table': 500,
 'possum': 500,
 'oak_tree': 500,
 'leopard': 500,
 'maple_tree': 500,
 'rabbit': 500,
 'chimpanzee': 500,
 'clock': 500,
 'streetcar': 500,
 'cockroach': 500,
 'snake': 500,
 'lobster': 500,
 'mountain': 500,
 'palm_tree': 500,
 'skyscraper': 500,
 'tractor': 500,
 'shark': 500,
 'butterfly': 500,
 'bottle': 500,
 'bee': 500,
 'chair': 500,
 'woman': 500,
 'hamster': 500,
 'otter': 500,
 'seal': 500,
 'lion': 500,
 'mushroom': 500,
 'girl': 500,
 'sweet_pepper': 500,
 'forest': 500,
 'crocodile': 500,
 'orange': 500,
 'tulip': 500,
 'mouse': 500,
 'camel': 500,
 'caterpillar': 500,
 'man': 500,
 'skunk': 500

In [7]:
test_classes_counts = {}

for image in test_set:
    label = test_set.classes[image[1]]
    if label not in test_classes_counts:
        test_classes_counts[label] = 1
    else:
        test_classes_counts[label] += 1
        
test_classes_counts

{'mountain': 100,
 'forest': 100,
 'seal': 100,
 'mushroom': 100,
 'sea': 100,
 'tulip': 100,
 'camel': 100,
 'butterfly': 100,
 'cloud': 100,
 'apple': 100,
 'skunk': 100,
 'streetcar': 100,
 'rocket': 100,
 'lamp': 100,
 'lion': 100,
 'wolf': 100,
 'rose': 100,
 'orange': 100,
 'dinosaur': 100,
 'chimpanzee': 100,
 'can': 100,
 'keyboard': 100,
 'bicycle': 100,
 'chair': 100,
 'plate': 100,
 'lawn_mower': 100,
 'turtle': 100,
 'palm_tree': 100,
 'shark': 100,
 'pickup_truck': 100,
 'boy': 100,
 'couch': 100,
 'house': 100,
 'porcupine': 100,
 'cockroach': 100,
 'clock': 100,
 'castle': 100,
 'beaver': 100,
 'bee': 100,
 'bottle': 100,
 'pear': 100,
 'baby': 100,
 'flatfish': 100,
 'oak_tree': 100,
 'leopard': 100,
 'snail': 100,
 'crocodile': 100,
 'rabbit': 100,
 'beetle': 100,
 'girl': 100,
 'sunflower': 100,
 'raccoon': 100,
 'train': 100,
 'ray': 100,
 'trout': 100,
 'bowl': 100,
 'snake': 100,
 'orchid': 100,
 'tractor': 100,
 'caterpillar': 100,
 'bus': 100,
 'mouse': 100,
 'cr

The train and test sets both feature the 100 classes to predict and are well-balanced, each class having 500 images in the train set and 100 images in the test set.

We decide to further split the train set in a train and a validation set. We will extract 10% of the train set images for that. This is done in the `CIFAR100DataModule`.

## Normalization

We create a function to compute the normalization statistics for our dataset. These statistics are computed on the train set and applied to the train, validation and test sets.
The function is called in the `setup` method of the `CIFAR100DataModule`.

In [ ]:
def compute_normstats(train_set):
    """
    Function to compute the normalization statistics on the train set.
    
    Takes in: train_set

    Returns: (red channel mean, green channel mean, blue channel mean), (red channel std, green channel std, blue channel std)
    """
    red_channels = torch.stack([train_set[i][0][0, :, :] for i in range(len(train_set))], dim=0)
    green_channels = torch.stack([train_set[i][0][1, :, :] for i in range(len(train_set))], dim=0)
    blue_channels = torch.stack([train_set[i][0][2, :, :] for i in range(len(train_set))], dim=0)
    train_set_mean = (red_channels.mean().item(), green_channels.mean().item(), blue_channels.mean().item())
    train_set_std = (red_channels.std().item(), green_channels.std().item(), blue_channels.std().item())
    return train_set_mean, train_set_std

## DataModule

Next, we create the DataModule. This class implements several methods to:
- download the dataset (if not already done)
- create the train, validation and test sets and apply all the necessary transforms (including data augmentation)
- create the dataloaders

In the `setup` method, we split the train set in a train and a validation set. This is done by picking 10% of the train images indices at random, after they are being shuffled. A seed is set prior to it and we ensured that it gave us a well-enough balanced validation set featuring all the 100 classes.

In [94]:
class CIFAR100DataModule(LightningDataModule):
    def __init__(self, batch_size=64, num_workers=2):
        super().__init__()
        self.batch_size = batch_size
        self.num_workers = num_workers
    
    
    def prepare_data(self):
        datasets.CIFAR100(root="./data", download=True, train=True)
        datasets.CIFAR100(root="./data", download=True, train=False)

        
    def setup(self, stage=None):
        train_set = datasets.CIFAR100(root="./data", train=True, transform=transforms.ToTensor())
        train_set_mean, train_set_std = compute_normstats(train_set)
        train_set_transforms = transforms.Compose([transforms.RandomCrop(32, padding=4),
                                            transforms.RandomHorizontalFlip(p=0.5),
                                            transforms.RandomRotation(degrees=15),
                                            transforms.ToTensor(),
                                            transforms.Normalize(train_set_mean, train_set_std, inplace=True)])

        validation_set_transforms = transforms.Compose([transforms.ToTensor(),
                                            transforms.Normalize(train_set_mean, train_set_std, inplace=True)])                                            

        test_set_transforms = transforms.Compose([transforms.ToTensor(),
                                            transforms.Normalize(train_set_mean, train_set_std, inplace=True)])

        
        # Get the train set images indices and shuffle them
        train_set_length = len(train_set)
        indices = list(range(train_set_length))
        np.random.seed(42)
        np.random.shuffle(indices)

        # Calculate the split point to have 10% of the train set as a validation set
        split = int(np.floor(0.9 * train_set_length))

        # Create a sampler for the train set (used in train_dataloader)
        self.train_sampler = SubsetRandomSampler(indices[:split])

        # Get the indices for the validation set (used in val_dataloader)
        self.validation_indices = indices[split:]

        # Create the train, validation and test sets
        self.cifar100_train = datasets.CIFAR100(root="./data", train=True, transform=train_set_transforms)
        self.cifar100_validation = datasets.CIFAR100(root="./data", train=True, transform=validation_set_transforms)
        self.cifar100_test = datasets.CIFAR100(root="./data", train=False, transform=test_set_transforms)

        # Retrieve classes from the train set
        self.classes = self.cifar100_train.classes
        
        
    def train_dataloader(self):
        cifar100_train = DataLoader(self.cifar100_train, batch_size=self.batch_size, sampler=self.train_sampler, num_workers=self.num_workers)
        return cifar100_train

    
    def val_dataloader(self):
        cifar100_validation = DataLoader(self.cifar100_validation, batch_size=self.batch_size, sampler=self.validation_indices, num_workers=self.num_workers)
        return cifar100_validation

    
    def test_dataloader(self):
        cifar100_test = DataLoader(self.cifar100_test, batch_size=self.batch_size, shuffle=False, num_workers=self.num_workers)
        return cifar100_test

## Model

Then, we define a function to create the model. Here, we take the `resnet18` architecture but not pre-trained.
We adapt it to our classification problem by modifying the first convolutional layer and the maxpool layer.

In [97]:
def create_model():
    """
    Function to create the model.
    
    Takes in: -

    Returns: model
    """
    model = torchvision.models.resnet18(pretrained=False, num_classes=100)
    model.conv1 = nn.Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    model.maxpool = nn.Identity()
    return model

## Lightning Pipeline

The Lightning pipeline is where everything happens. It implements the following methods:
- `configure_optimizers`
- `forward`
- `training_step`
- `training_epoch_end`
- `validation_step`
- `validation_epoch_end`
- `test_step`
- `test_epoch_end`
- `on_save_checkpoint`

In [100]:
class CIFAR100ResNet(LightningModule):
    def __init__(self, learning_rate, batch_size):
        super().__init__()
        
        # Save hyperparameters to the checkpoint
        self.save_hyperparameters() 

        self.confmat = ConfusionMatrix(num_classes=100)
        
        # Creation of the model
        self.model = create_model()

        # Instantiation of the number of classes
        self.n_classes = 100 
   
        # Instantiation of the learning rate
        self.learning_rate = learning_rate

        # Instantiation of the batch_size
        self.batch_size = batch_size
        
    def configure_optimizers(self): 
        optimizer = torch.optim.SGD(
            self.parameters(),
            lr=self.hparams.learning_rate,
            momentum=0.9,
            weight_decay=5e-4,
        )
        scheduler_dict = {
            "scheduler": MultiStepLR(
                optimizer,
                milestones=[60,120,160],
                gamma=0.2
            ),
            "interval": "epoch"
        }        
        #steps_per_epoch = int(np.ceil(45000 / self.batch_size))
        #scheduler_dict = {
            #"scheduler": OneCycleLR(
            #    optimizer,
            #    max_lr=0.1,
            #    epochs=self.trainer.max_epochs,
            #    steps_per_epoch=steps_per_epoch
            #),
            #"interval": "step"
        #}
        return {"optimizer": optimizer, "lr_scheduler": scheduler_dict}
    
    def forward(self, x):
        out = self.model(x)
        return F.log_softmax(out, dim=1)

    def training_step(self, batch, batch_idx):
        x, y = batch
        logits = self(x)
        loss = F.nll_loss(logits, y)
        preds = torch.argmax(logits, dim=1)
        self.log("train_loss", loss, on_step=True, on_epoch=True)
        return {"inputs":x, "targets":y, "predictions":preds, "loss":loss}    

    def training_epoch_end(self, outputs):
        # Log weights and biases for all layers of the model
        for name, params in self.named_parameters():
            self.logger.experiment.add_histogram(name, params,self.current_epoch)
        # Only after the first training epoch, log one of the training inputs as a figure and log the model graph
        if self.current_epoch == 0:
            image_samples = outputs[0]["inputs"][:10]
            image_samples = image_samples.cpu()
            image_samples_grid = make_grid(image_samples, normalize=True)
            image_samples_grid = image_samples_grid.numpy()
            fig = plt.figure()
            ax = fig.add_subplot(111)
            ax.imshow(np.transpose(image_samples_grid, (1, 2, 0)))
            self.logger.experiment.add_figure(f"Training sample normalized images", fig)
            input_sample = outputs[0]["inputs"][0]
            input_sample = torch.unsqueeze(input_sample, 3)
            input_sample = torch.permute(input_sample, (3,0,1,2))
            self.logger.experiment.add_graph(self, input_sample)

    def validation_step(self, batch, batch_idx):
        x, y = batch
        logits = self(x)
        loss = F.nll_loss(logits, y)
        preds = torch.argmax(logits, dim=1)
        acc = accuracy(preds, y)
        self.log(f"validation_loss", loss, prog_bar=True)
        self.log(f"validation_acc", acc, prog_bar=True)
        return {"inputs":x, "targets":y, "predictions":preds, "loss":loss} 

    def validation_epoch_end(self, outputs):
        # Concatenate the targets of all batches
        targets = torch.cat([output["targets"] for output in outputs])
        # Concatenate the predictions of all batches
        preds = torch.cat([output["predictions"] for output in outputs])
        # Compute the confusion matrix
        cm = self.confmat(preds, targets)
        # Send it to the CPU
        cm = cm.cpu()
        # For each class
        for class_id in range(self.n_classes):
                # Calculate and log its prediction precision on the full validation set
                precision = cm[class_id, class_id] / torch.sum(cm[:,class_id])
                precision = round(precision.item()*100,1)
                self.log(f"validation_precision/{self.n_classes}", precision)

    def test_step(self, batch, batch_idx):
        x, y = batch
        logits = self(x)
        loss = F.nll_loss(logits, y)
        preds = torch.argmax(logits, dim=1)
        acc = accuracy(preds, y)
        self.log(f"test_loss", loss, prog_bar=True)
        self.log(f"test_acc", acc, prog_bar=True)
        return {"targets":y, "predictions":preds}

    def test_epoch_end(self, outputs):
        targets = torch.cat([output["targets"] for output in outputs])
        preds = torch.cat([output["predictions"] for output in outputs])
        # Compute the total prediction accuracy on the full test set
        acc = accuracy(preds, targets)
        # Compute the confusion matrix
        cm = self.confmat(preds, targets)
        # Send it to the CPU
        cm = cm.cpu()
        # Write the test set prediction performances to an output file
        with open("test_set_predictions.txt", "w") as f:
            f.write("==================================================\n")
            f.write("ACCURACY\n")
            f.write("==================================================\n")
            f.write("\n")            
            f.write(f"Total: {round(acc.item()*100, 1)}%\n")
            f.write("\n")
            f.write("Per Class:\n")
            f.write("Class - Accuracy (%)\n")
            for class_id in range(self.n_classes):
                precision = cm[class_id, class_id] / torch.sum(cm[:,class_id])            
                precision = round(precision.item()*100, 1)
                f.write(f"{self.trainer.datamodule.classes[class_id]} - {precision}\n")
            f.write("\n")
            f.write("\n")
            f.write("==================================================\n")
            f.write("PREDICTIONS DETAIL\n")
            f.write("==================================================\n")
            f.write("Image index - Target class - Predicted class\n")
            # Write the target class and the predicted class for each test image
            for i in range(len(targets)):
                f.write(f"{i} - {self.trainer.datamodule.classes[targets[i]]} - {self.trainer.datamodule.classes[preds[i]]}\n")
        
    def on_save_checkpoint(self, checkpoint):
        # Get the state_dict from self.model to get rid of the "model." prefix
        checkpoint["state_dict"] = self.state_dict()

## Model Training and Evaluation

Finally, we set some of our hyper-parameters, instantiate the classes defined above as well as some callbacks (tensorboard logger, learning rate monitor, early stopping and checkpoint saving), and train and test our model.

In [ ]:
# Filter harmless waarnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings("ignore", ".*Your `val_dataloader` has `shuffle=True`.*")
warnings.filterwarnings("ignore", ".*Checkpoint directory.*")

# Print name of graphics card used
gpus = min(0, torch.cuda.device_count())

# Set number of workers (for dataloaders)
num_workers = int(os.cpu_count() / 3)
print(f"Number of workers used: {num_workers}")

# Set maximum number of epochs to train for
max_epochs = 200
print(f"Maximum number of epochs: {max_epochs}")

# Set the batch size
batch_size = 256 if args.accelerator else 64
print(f"Batch size: {batch_size}")

# Set the initial learning rate
learning_rate = 0.1
print(f"Initial learning rate: {learning_rate}")    

# Instantiate the DataModule
dm = datamodule.CIFAR100DataModule(batch_size=batch_size, num_workers=num_workers)

# Instantiate the logger
tensorboard_logger = TensorBoardLogger(save_dir="logs")

# Instantiate early stopping based on epoch validation loss
early_stopping = EarlyStopping("validation_loss", patience=20, verbose=True)

# Instantiate a learning rate monitor
lr_monitor = LearningRateMonitor(logging_interval='step')

# Instantiate a checkpoint callback
checkpoint = ModelCheckpoint(
                            dirpath=f"./checkpoints/",
                            filename="{epoch}-{validation_loss:.2f}",
                            verbose=True,
                            monitor="validation_loss",
                            save_last = False,
                            save_top_k=1,      
                            mode="min",
                            save_weights_only=True
                            )

# Instantiate the trainer
trainer = Trainer(
                gpus=gpus,
                max_epochs=max_epochs, 
                logger=tensorboard_logger,
                log_every_n_steps = 1,
                callbacks=[lr_monitor, checkpoint]
                ) 

# Instantiate the pipeline
pipeline = pipeline.CIFAR100ResNet(learning_rate=learning_rate, batch_size=batch_size)  
    
# Fit the trainer on the training set
trainer.fit(pipeline, dm)

# Test on the test set
trainer.test(pipeline, dm)

/Users/antoine/opt/anaconda3/lib/python3.8/site-packages/pytorch_lightning/trainer/connectors/callback_connector.py:90: LightningDeprecationWarning: Setting `Trainer(progress_bar_refresh_rate=10)` is deprecated in v1.5 and will be removed in v1.7. Please pass `pytorch_lightning.callbacks.progress.TQDMProgressBar` with `refresh_rate` directly to the Trainer's `callbacks` argument instead. Or, to disable the progress bar pass `enable_progress_bar = False` to the Trainer.
  rank_zero_deprecation(
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs

  | Name  | Type   | Params
---------------------------------
0 | model | ResNet | 11.2 M
---------------------------------
11.2 M    Trainable params
0         Non-trainable params
11.2 M    Total params
44.881    Total estimated model params size (MB)
/Users/antoine/opt/anaconda3/lib/python3.8/site-packages/pytorch_lightning/trainer/data_loading.py:132: UserWarning: The dataloader, tr

Training: 0it [00:00, ?it/s]

The model's performance metrics and the evolution of its hyperparameters, as well as images samples from the train set, can be visualised on the tensorboard.

In [ ]:
%load_ext tensorboard
%tensorboard --logdir logs